# Trendyol Mila Chatbot OpenAI API Entegrasyonu

Bu notebook, hazırlanan veri seti üzerinde OpenAI **gpt-5-nano** ile **Structured JSON Schema** (Pydantic) kullanarak analiz yapar.

## 1. Gerekli Kütüphaneler ve API Kurulumu

In [1]:
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import time
from datetime import datetime
import os

# OpenAI API
from openai import OpenAI

# Pydantic için structured output
from pydantic import BaseModel

# Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Kütüphaneler yüklendi! (Pydantic dahil)")

✅ Kütüphaneler yüklendi! (Pydantic dahil)


## 2. Kütüphane ve Sürüm Kontrolü

In [2]:
# KÜTÜPHANE VE SÜRÜM KONTROLÜ
print("🔍 KÜTÜPHANE KONTROLÜ:")
print("=" * 40)

try:
    import openai
    print(f"✅ OpenAI version: {openai.__version__}")
except Exception as e:
    print(f"❌ OpenAI error: {e}")

try:
    from pydantic import BaseModel
    import pydantic
    print(f"✅ Pydantic version: {pydantic.__version__}")
except Exception as e:
    print(f"❌ Pydantic error: {e}")

try:
    from openai import OpenAI
    test_client = OpenAI(api_key="test")
    if hasattr(test_client.chat.completions, 'parse'):
        print("✅ OpenAI parse() metodu mevcut")
    else:
        print("❌ OpenAI parse() metodu YOK - eski sürüm!")
except Exception as e:
    print(f"❌ OpenAI parse kontrolü: {e}")

print("\n🧪 BASIT PYDANTİC TEST:")
class TestModel(BaseModel):
    name: str

try:
    test = TestModel(name="test")
    print(f"✅ Pydantic çalışıyor: {test.model_dump()}")
except Exception as e:
    print(f"❌ Pydantic hatası: {e}")

🔍 KÜTÜPHANE KONTROLÜ:
✅ OpenAI version: 1.106.1
✅ Pydantic version: 2.11.7
✅ OpenAI parse() metodu mevcut

🧪 BASIT PYDANTİC TEST:
✅ Pydantic çalışıyor: {'name': 'test'}


## 3. OpenAI API Key Ayarları

In [ ]:
# OpenAI API Key ayarlama - GÜVENLİK UYARISI: API key'i çevresel değişkenden alın
API_KEY = os.getenv('OPENAI_API_KEY')

if not API_KEY:
    print("⚠️ OPENAI_API_KEY çevresel değişkeni tanımlı değil!")
    print("Terminal'de şu komutu çalıştırın: export OPENAI_API_KEY='your-openai-api-key-here'")
    # Geçici olarak direkt key kullanımı (production'da YAPMAYIN!)
    API_KEY = "your-openai-api-key-here"

# OpenAI Client oluştur
client = OpenAI(api_key=API_KEY)

print("✅ OpenAI API bağlantısı kuruldu")

## 4. Model Testi

In [4]:
# OpenAI API gpt-5-nano test
print("🧪 gpt-5-nano modelini test ediyoruz...")

try:
    test_response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "user", "content": "Merhaba, API çalışıyor mu?"}
        ],
        max_completion_tokens=50
    )
    
    print("✅ gpt-5-nano BAŞARILI!")
    print(f"Test yanıtı: {test_response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ API bağlantı hatası: {e}")
    print("\nOlası çözümler:")
    print("1. API key'inizi kontrol edin")
    print("2. İnternet bağlantınızı kontrol edin")
    print("3. OpenAI quota'nızı kontrol edin")

🧪 gpt-5-nano modelini test ediyoruz...
✅ gpt-5-nano BAŞARILI!
Test yanıtı: 


## 5. Hazırlanan Verileri Yükleme

In [5]:
# Hazırlanan verileri yükle
with open('prepared_data.json', 'r', encoding='utf-8') as f:
    prepared_data = json.load(f)

print("📊 Yüklenen veri seti:")
print(f"  Sentiment analizi için: {len(prepared_data['sentiment_data'])} sohbet")
print(f"  Intent sınıflandırması için: {len(prepared_data['intent_data'])} sohbet")
print(f"  Çözüm durumu tahmini için: {len(prepared_data['resolution_data'])} sohbet")

📊 Yüklenen veri seti:
  Sentiment analizi için: 40 sohbet
  Intent sınıflandırması için: 40 sohbet
  Çözüm durumu tahmini için: 40 sohbet


## 6. Pydantic Modelleri - Structured JSON Schema

In [6]:
# Pydantic modelleri - Structured JSON Schema için
class SentimentAnalysis(BaseModel):
    sentiment: str  # "Pozitif", "Negatif", "Nötr"
    explanation: str  # Açıklama

class IntentAnalysis(BaseModel):
    intent: str  # Ana kategori
    detail: str  # Spesifik detay
    category: str  # Tür: Şikayet/Sorun/Soru/İstek

class ResolutionAnalysis(BaseModel):
    status: str  # "Çözüldü", "Çözülemedi" 
    explanation: str  # Açıklama
    confidence: str  # "Yüksek", "Orta", "Düşük"

print("✅ Pydantic modelleri tanımlandı!")

✅ Pydantic modelleri tanımlandı!


## 7. OpenAI Analyzer Sınıfı - Structured Output ile

In [7]:
class OpenAIAnalyzer:
    """
    OpenAI API ile sohbet analizi yapan sınıf - Structured JSON Schema ile
    """
    
    def __init__(self, client: OpenAI):
        self.client = client
        self.model = "gpt-5-nano"
        self.results = {}
    
    def analyze_sentiment(self, conversation: str, chat_id: str) -> Dict:
        """
        Structured output ile sentiment analizi
        """
        system_prompt = """Sen Trendyol müşteri hizmetleri uzmanısın. Sohbetlerin sentiment'ini analiz ediyorsun.

SENTIMENT KURALLARI:
• Pozitif: Müşteri teşekkür ediyor, memnun, sorun çözüldü, "harika", "teşekkürler" gibi kelimeler
• Negatif: Müşteri kızgın, şikayet ediyor, "berbat", "kötü", "sinirli" gibi kelimeler  
• Nötr: Normal soru-cevap, ne pozitif ne negatif

Sohbeti analiz et ve sentiment ile kısa açıklama ver."""
        
        try:
            completion = self.client.chat.completions.parse(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Sohbet:\n{conversation}"}
                ],
                response_format=SentimentAnalysis,
                max_completion_tokens=5000
            )
            
            result = completion.choices[0].message.parsed
            result_dict = result.model_dump()
            
            return {
                'chat_id': chat_id,
                'predicted_sentiment': result_dict['sentiment'],
                'explanation': result_dict['explanation'],
                'raw_response': str(result_dict)
            }
            
        except Exception as e:
            print(f"❌ Sentiment analizi hatası (Chat {chat_id}): {e}")
            return {
                'chat_id': chat_id,
                'predicted_sentiment': 'Nötr',
                'explanation': f'Hata: {str(e)}',
                'raw_response': ''
            }
    
    def analyze_intent(self, conversation: str, chat_id: str) -> Dict:
        """
        Structured output ile intent analizi
        """
        system_prompt = """Sen Trendyol müşteri hizmetleri uzmanısın. Müşterinin asıl niyetini buluyorsun.

INTENT KATEGORİLERİ:
• İade: Ürün iade etmek istiyor
• Ödeme: Ödeme sorunu, para kesimi, kart problemi  
• Kargo: Teslimat sorunu, geç geldi, kayıp
• İptal: Sipariş iptal etmek istiyor
• Eksik ürün: Siparişte eksik ürün var
• Ürün sorunu: Bozuk/kusurlu ürün geldi
• Hesap: Hesap ayarları, profil sorunu
• Şifre sıfırlama: Şifre unuttu, giriş yapamıyor
• Destek: Genel soru, bilgi alma

TÜR KATEGORİLERİ:
• Şikayet: Memnuniyetsizlik dile getiriyor
• Sorun: Teknik/operasyonel problem var
• Soru: Bilgi öğrenmek istiyor
• İstek: Bir işlem yapmasını istiyor

Sohbeti analiz et ve müşterinin intent'ini, detayını ve türünü belirle."""
        
        try:
            completion = self.client.chat.completions.parse(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Sohbet:\n{conversation}"}
                ],
                response_format=IntentAnalysis,
                max_completion_tokens=5000
            )
            
            result = completion.choices[0].message.parsed
            result_dict = result.model_dump()
            
            return {
                'chat_id': chat_id,
                'predicted_intent': result_dict['intent'],
                'predicted_detail': result_dict['detail'],
                'predicted_category': result_dict['category'],
                'raw_response': str(result_dict)
            }
            
        except Exception as e:
            print(f"❌ Intent analizi hatası (Chat {chat_id}): {e}")
            return {
                'chat_id': chat_id,
                'predicted_intent': 'Destek',
                'predicted_detail': f'Hata: {str(e)}',
                'predicted_category': 'Soru',
                'raw_response': ''
            }
    
    def predict_resolution(self, conversation: str, chat_id: str) -> Dict:
        """
        Structured output ile çözüm durumu tahmini
        """
        system_prompt = """Sen Trendyol müşteri hizmetleri uzmanısın. Sohbetin çözülüp çözülmediğini değerlendiriyorsun.

ÇÖZÜM DURUMU KURALLARI (GELİŞTİRİLMİŞ):
  • Çözüldü: 
    - Somut aksiyon alındı (ürün gönderildi, para iadesi yapıldı, vb.) VEYA
    - Müşteri bilgi istedi ve tatmin edici bilgi verildi + müşteri teşekkür etti VEYA
    - Müşteri sorunu çözüldü dedi, "tamam", "anladım", "teşekkürler" gibi memnun ifadeler kullandı

  • Çözülemedi:
    - Sorun devam ediyor ve hiçbir aksiyon alınmadı VEYA
    - Müşteri hala şikayet ediyor, tatmin olmamış VEYA
    - Sadece "bekleyin", "kontrol edeceğiz" gibi belirsiz yanıtlar verildi

  ÖNEMLİ: Bilgi verme de çözümdür! Müşteri iade prosedürü sordu ve bilgi verildi + teşekkür etti = Çözüldü

  GÜVEN SEVİYELERİ:
  • Yüksek: Net çözüm/çözümsüzlük belli
  • Orta: Çoğunlukla belli ama kesin değil  
  • Düşük: Belirsiz durum

Sohbeti analiz et ve çözüm durumunu, açıklamasını ve güven seviyesini belirle."""
        
        try:
            completion = self.client.chat.completions.parse(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Sohbet:\n{conversation}"}
                ],
                response_format=ResolutionAnalysis,
                max_completion_tokens=10000
            )
            
            result = completion.choices[0].message.parsed
            result_dict = result.model_dump()
            
            return {
                'chat_id': chat_id,
                'predicted_resolution': result_dict['status'],
                'explanation': result_dict['explanation'],
                'confidence': result_dict['confidence'],
                'raw_response': str(result_dict)
            }
            
        except Exception as e:
            print(f"❌ Çözüm durumu analizi hatası (Chat {chat_id}): {e}")
            return {
                'chat_id': chat_id,
                'predicted_resolution': 'Çözülemedi',
                'explanation': f'Hata: {str(e)}',
                'confidence': 'Düşük',
                'raw_response': ''
            }

# Analyzer'ı başlat
analyzer = OpenAIAnalyzer(client)
print("🤖 OpenAI Analyzer hazır! (Structured JSON Schema ile)")

🤖 OpenAI Analyzer hazır! (Structured JSON Schema ile)


## 8. API Test - Tek Sohbet Analizi

In [8]:
# STRUCTURED JSON SCHEMA TEST - YENİ ÇÖZÜM
print("🧪 YENİ STRUCTURED OUTPUT TESTİ:")
print("=" * 60)

# Test sohbeti
test_chat = prepared_data['sentiment_data'][0]
print(f"Test Sohbeti: {test_chat['sohbet_id']}")
print(f"Gerçek Sentiment: {test_chat['ground_truth_sentiment']}")
print(f"\nSohbet içeriği:\n{test_chat['conversation']}")

# Yeni structured output ile test
sentiment_result = analyzer.analyze_sentiment(
    test_chat['conversation'], 
    str(test_chat['sohbet_id'])
)

print(f"\n📊 SENTIMENT SONUCU:")
print(f"Gerçek: {test_chat['ground_truth_sentiment']}")
print(f"Tahmin: {sentiment_result['predicted_sentiment']}")
print(f"Açıklama: {sentiment_result['explanation']}")
print(f"Ham Yanıt: {sentiment_result['raw_response']}")
print(f"Doğru mu: {'✅' if sentiment_result['predicted_sentiment'] == test_chat['ground_truth_sentiment'] else '❌'}")

🧪 YENİ STRUCTURED OUTPUT TESTİ:
Test Sohbeti: 90001
Gerçek Sentiment: Pozitif

Sohbet içeriği:
Bot: Merhaba! Ben AI Asistan. Size nasıl yardımcı olabilirim?
Müşteri: Siparişimde bir ürün eksik geldi. Ne yapmalıyım?
Bot: Sipariş numaranızı alabilir miyim?
Müşteri: 98765432
Bot: Kontrol ediyorum... Siparişinizde bir hata olduğunu görüyorum. Yeni ürününüz en kısa sürede gönderilecektir. Özür dileriz.
Müşteri: Harika, teşekkür ederim!
Bot: Rica ederim, iyi günler dileriz.


KeyboardInterrupt: 

In [ ]:
# Intent analizi test
print("🎯 INTENT ANALİZİ TESTİ:")
print("=" * 60)

test_intent = prepared_data['intent_data'][0]
print(f"Test Sohbeti: {test_intent['sohbet_id']}")
print(f"Gerçek Intent: {test_intent['ground_truth_intent']}")

intent_result = analyzer.analyze_intent(
    test_intent['conversation'], 
    str(test_intent['sohbet_id'])
)

print(f"\n📊 INTENT SONUCU:")
print(f"Gerçek Intent: {test_intent['ground_truth_intent']}")
print(f"Tahmin Intent: {intent_result['predicted_intent']}")
print(f"Gerçek Detay: {test_intent['ground_truth_intent_detay']}")
print(f"Tahmin Detay: {intent_result['predicted_detail']}")
print(f"Tahmin Tür: {intent_result['predicted_category']}")
print(f"Ham Yanıt: {intent_result['raw_response']}")
print(f"Doğru mu: {'✅' if intent_result['predicted_intent'] == test_intent['ground_truth_intent'] else '❌'}")

🎯 INTENT ANALİZİ TESTİ:
Test Sohbeti: 90001
Gerçek Intent: Eksik ürün

📊 INTENT SONUCU:
Gerçek Intent: Eksik ürün
Tahmin Intent: Eksik ürün
Gerçek Detay: Sipariş teslimatında eksik ürün
Tahmin Detay: Müşteri, siparişinde bir ürün eksik geldiğini bildirdi ve nasıl ilerlemesi gerektiğini sordu. Sipariş numarası 98765432 verildi. Bot durumı inceleyerek eksik ürün bulunduğunu ve yeni ürünün en kısa sürede gönderileceğini belirtti.
Tahmin Tür: Soru
Ham Yanıt: {'intent': 'Eksik ürün', 'detail': 'Müşteri, siparişinde bir ürün eksik geldiğini bildirdi ve nasıl ilerlemesi gerektiğini sordu. Sipariş numarası 98765432 verildi. Bot durumı inceleyerek eksik ürün bulunduğunu ve yeni ürünün en kısa sürede gönderileceğini belirtti.', 'category': 'Soru'}
Doğru mu: ✅


In [ ]:
# Çözüm durumu tahmini test
print("🔧 ÇÖZÜM DURUMU TAHMİNİ TESTİ:")
print("=" * 60)

test_resolution = prepared_data['resolution_data'][0]
print(f"Test Sohbeti: {test_resolution['sohbet_id']}")
print(f"Gerçek Durum: {test_resolution['ground_truth_resolution']}")

resolution_result = analyzer.predict_resolution(
    test_resolution['conversation'], 
    str(test_resolution['sohbet_id'])
)

print(f"\n📊 ÇÖZÜM SONUCU:")
print(f"Gerçek Durum: {test_resolution['ground_truth_resolution']}")
print(f"Tahmin Durum: {resolution_result['predicted_resolution']}")
print(f"Açıklama: {resolution_result['explanation']}")
print(f"Güven Düzeyi: {resolution_result['confidence']}")
print(f"Ham Yanıt: {resolution_result['raw_response']}")
print(f"Doğru mu: {'✅' if resolution_result['predicted_resolution'] == test_resolution['ground_truth_resolution'] else '❌'}")

🔧 ÇÖZÜM DURUMU TAHMİNİ TESTİ:
Test Sohbeti: 90001
Gerçek Durum: Çözüldü

📊 ÇÖZÜM SONUCU:
Gerçek Durum: Çözüldü
Tahmin Durum: Çözüldü
Açıklama: Müşterinin siparişindeki eksik ürün için yeni ürünün gönderileceği bilgisi verildi ve müşteri bunu memnuniyetle karşıladı (teşekkür etti). Bu memnun ifade ve somut aksiyon (gönderim) durumunu çözüldü olarak değerlendirir.
Güven Düzeyi: Yüksek
Ham Yanıt: {'status': 'Çözüldü', 'explanation': 'Müşterinin siparişindeki eksik ürün için yeni ürünün gönderileceği bilgisi verildi ve müşteri bunu memnuniyetle karşıladı (teşekkür etti). Bu memnun ifade ve somut aksiyon (gönderim) durumunu çözüldü olarak değerlendirir.', 'confidence': 'Yüksek'}
Doğru mu: ✅


## 9. Toplu Analiz Fonksiyonu

In [ ]:
# Tüm 40 sohbet için toplu analiz fonksiyonu - GÜNCELLENDİ
def run_full_batch_analysis(analyzer: OpenAIAnalyzer, data_subset: List[Dict], analysis_type: str):
    """
    Tüm sohbetler için analiz çalıştırır (40 sohbet)
    """
    results = []
    total_chats = len(data_subset)
    
    print(f"🚀 {analysis_type} analizi başlatılıyor... (Toplam {total_chats} sohbet)")
    print("=" * 60)
    
    for i, chat_data in enumerate(data_subset):
        print(f"İşleniyor: {i+1}/{total_chats} - Chat ID: {chat_data['sohbet_id']}")
        
        chat_id = str(chat_data['sohbet_id'])
        conversation = chat_data['conversation']
        
        try:
            if analysis_type == "sentiment":
                result = analyzer.analyze_sentiment(conversation, chat_id)
                result['ground_truth'] = chat_data['ground_truth_sentiment']
                
            elif analysis_type == "intent":
                result = analyzer.analyze_intent(conversation, chat_id)
                result['ground_truth_intent'] = chat_data['ground_truth_intent']
                result['ground_truth_detail'] = chat_data['ground_truth_intent_detay']
                result['ground_truth_tur'] = chat_data['ground_truth_tur']
                
            elif analysis_type == "resolution":
                result = analyzer.predict_resolution(conversation, chat_id)
                result['ground_truth'] = chat_data['ground_truth_resolution']
            
            results.append(result)
            
            # İlerleme göstergesi
            if (i + 1) % 10 == 0:
                print(f"✅ {i + 1} sohbet tamamlandı!")
                
        except Exception as e:
            print(f"❌ Hata (Chat {chat_id}): {e}")
            # Hata durumunda varsayılan sonuç ekle
            error_result = {
                'chat_id': chat_id,
                'error': str(e)
            }
            if analysis_type == "sentiment":
                error_result['predicted_sentiment'] = 'Nötr'
                error_result['explanation'] = f'Hata: {str(e)}'
                error_result['ground_truth'] = chat_data['ground_truth_sentiment']
            elif analysis_type == "intent":
                error_result['predicted_intent'] = 'Destek'
                error_result['predicted_detail'] = f'Hata: {str(e)}'
                error_result['predicted_category'] = 'Soru'
                error_result['ground_truth_intent'] = chat_data['ground_truth_intent']
                error_result['ground_truth_detail'] = chat_data['ground_truth_intent_detay']
                error_result['ground_truth_tur'] = chat_data['ground_truth_tur']
            elif analysis_type == "resolution":
                error_result['predicted_resolution'] = 'Çözülemedi'
                error_result['explanation'] = f'Hata: {str(e)}'
                error_result['confidence'] = 'Düşük'
                error_result['ground_truth'] = chat_data['ground_truth_resolution']
            results.append(error_result)
        
        # Rate limiting - API limits için bekleme
        time.sleep(1)
    
    print(f"✅ {analysis_type} analizi tamamlandı! ({total_chats} sohbet)")
    return results

print("✅ Tüm 40 sohbet için güncellenmiş toplu analiz fonksiyonu hazır!")

## 10. Sınırlı Batch Analizi - Sentiment

In [ ]:
# İlk 5 sohbet için sentiment analizi
sentiment_results = run_batch_analysis(
    analyzer, 
    prepared_data['sentiment_data'], 
    "sentiment", 
    max_requests=5
)

🚀 sentiment analizi başlatılıyor... (İlk 5 sohbet)
İşleniyor: 1/5
İşleniyor: 2/5
İşleniyor: 3/5
İşleniyor: 4/5
İşleniyor: 5/5
✅ sentiment analizi tamamlandı!


In [ ]:
# Sentiment analizi sonuçlarını değerlendir
print("📊 SENTIMENT ANALİZİ SONUÇLARI:")
print("=" * 50)

correct_predictions = 0
for result in sentiment_results:
    is_correct = result['predicted_sentiment'] == result['ground_truth']
    if is_correct:
        correct_predictions += 1
    
    print(f"Chat {result['chat_id']}: {result['ground_truth']} → {result['predicted_sentiment']} {'✅' if is_correct else '❌'}")
    print(f"  Açıklama: {result['explanation']}")
    print(f"  Ham Yanıt: {result['raw_response'][:100]}...")
    print()

sentiment_accuracy = (correct_predictions / len(sentiment_results)) * 100
print(f"🎯 Sentiment Doğruluk Oranı: {sentiment_accuracy:.1f}% ({correct_predictions}/{len(sentiment_results)})")

📊 SENTIMENT ANALİZİ SONUÇLARI:
Chat 90001: Pozitif → Pozitif ✅
  Açıklama: Müşterinin eksik ürün sorunu çözüldü; teşekkür etti ve olumlu geri dönüş sağlandı.
  Ham Yanıt: {'sentiment': 'Pozitif', 'explanation': 'Müşterinin eksik ürün sorunu çözüldü; teşekkür etti ve olum...

Chat 90002: Negatif → Negatif ✅
  Açıklama: Kullanıcı hesap giriş problemi yaşıyor, şifre sıfırlama linkinin gelmediğini belirtiyor ve 'bu durum çok can sıkıcı' diyerek rahatsızlık ve memnuniyetsizlik ifade ediyor.
  Ham Yanıt: {'sentiment': 'Negatif', 'explanation': "Kullanıcı hesap giriş problemi yaşıyor, şifre sıfırlama lin...

Chat 90003: Nötr → Pozitif ❌
  Açıklama: Müşteri teşekkür etti ve bilgilendirilmeyi teyit etti; süreç netleşti ve olumlu bir iletişim sağlandı.
  Ham Yanıt: {'sentiment': 'Pozitif', 'explanation': 'Müşteri teşekkür etti ve bilgilendirilmeyi teyit etti; süre...

Chat 90004: Negatif → Negatif ✅
  Açıklama: Müşteri kupon kodunun çalışmaması nedeniyle sorun bildiriyor ve denenen çözümlere rağ

## 11. Sınırlı Batch Analizi - Intent

In [ ]:
# İlk 3 sohbet için intent analizi (daha az çünkü daha uzun)
intent_results = run_batch_analysis(
    analyzer, 
    prepared_data['intent_data'], 
    "intent", 
    max_requests=3
)

🚀 intent analizi başlatılıyor... (İlk 3 sohbet)
İşleniyor: 1/3
İşleniyor: 2/3
İşleniyor: 3/3
✅ intent analizi tamamlandı!


In [ ]:
# Intent analizi sonuçlarını değerlendir
print("🎯 INTENT ANALİZİ SONUÇLARI:")
print("=" * 50)

intent_correct = 0
for result in intent_results:
    is_correct = result['predicted_intent'] == result['ground_truth_intent']
    if is_correct:
        intent_correct += 1
    
    print(f"Chat {result['chat_id']}:")
    print(f"  Gerçek: {result['ground_truth_intent']} | Tahmin: {result['predicted_intent']} {'✅' if is_correct else '❌'}")
    print(f"  Detay: {result['predicted_detail']}")
    print(f"  Tür: {result['predicted_category']}")
    print(f"  Ham Yanıt: {result['raw_response'][:100]}...")
    print()

intent_accuracy = (intent_correct / len(intent_results)) * 100
print(f"🎯 Intent Doğruluk Oranı: {intent_accuracy:.1f}% ({intent_correct}/{len(intent_results)})")

🎯 INTENT ANALİZİ SONUÇLARI:
Chat 90001:
  Gerçek: Eksik ürün | Tahmin: Eksik ürün ✅
  Detay: Sipariş numarası 98765432'de eksik ürün bildirildi; müşteri 'Ne yapmalıyım?' diye bilgi talep etti. Sistem kontrolü sonrası hatanın tespit edildi ve eksik ürün için yeni ürün gönderileceği bilgisi paylaşıldı.
  Tür: Soru
  Ham Yanıt: {'intent': 'Eksik ürün', 'detail': "Sipariş numarası 98765432'de eksik ürün bildirildi; müşteri 'Ne ...

Chat 90002:
  Gerçek: Şifre sıfırlama | Tahmin: Şifre sıfırlama ✅
  Detay: Hesabına giriş yapamıyor; şifremi unuttum. Şifre sıfırlama linki için e-posta adresini girdim; fakat link gelmiyor. E-posta kutusunu spam klasörü dahil kontrol ettim; hiçbir yerde yok. Başka bir yol yok mu? Alternatif yöntemler (SMS/telefon doğrulama veya manuel sıfırlama) talep ediyor.
  Tür: Sorun
  Ham Yanıt: {'intent': 'Şifre sıfırlama', 'detail': 'Hesabına giriş yapamıyor; şifremi unuttum. Şifre sıfırlama ...

Chat 90003:
  Gerçek: İade | Tahmin: İade ✅
  Detay: İade sürecinin nasıl 

## 12. Sınırlı Batch Analizi - Resolution

In [ ]:
# İlk 3 sohbet için çözüm durumu analizi
resolution_results = run_batch_analysis(
    analyzer, 
    prepared_data['resolution_data'], 
    "resolution", 
    max_requests=3
)

🚀 resolution analizi başlatılıyor... (İlk 3 sohbet)
İşleniyor: 1/3
İşleniyor: 2/3
İşleniyor: 3/3
✅ resolution analizi tamamlandı!


In [ ]:
# Çözüm durumu analizi sonuçlarını değerlendir
print("🔧 ÇÖZÜM DURUMU ANALİZİ SONUÇLARI:")
print("=" * 50)

resolution_correct = 0
for result in resolution_results:
    is_correct = result['predicted_resolution'] == result['ground_truth']
    if is_correct:
        resolution_correct += 1
    
    print(f"Chat {result['chat_id']}:")
    print(f"  Gerçek: {result['ground_truth']} | Tahmin: {result['predicted_resolution']} {'✅' if is_correct else '❌'}")
    print(f"  Açıklama: {result['explanation']}")
    print(f"  Güven: {result['confidence']}")
    print(f"  Ham Yanıt: {result['raw_response'][:100]}...")
    print()

resolution_accuracy = (resolution_correct / len(resolution_results)) * 100
print(f"🎯 Çözüm Durumu Doğruluk Oranı: {resolution_accuracy:.1f}% ({resolution_correct}/{len(resolution_results)})")

🔧 ÇÖZÜM DURUMU ANALİZİ SONUÇLARI:
Chat 90001:
  Gerçek: Çözüldü | Tahmin: Çözüldü ✅
  Açıklama: Eksik ürün sorunu doğrulandı; hatalı/eksik ürün için düzeltici işlem başlatıldı ve yerine yeni ürün en kısa sürede gönderilecek, müşteri teşekkür etti ve memnun görünüyor.
  Güven: Yüksek
  Ham Yanıt: {'status': 'Çözüldü', 'explanation': 'Eksik ürün sorunu doğrulandı; hatalı/eksik ürün için düzeltici...

Chat 90002:
  Gerçek: Çözülemedi | Tahmin: Çözülemedi ✅
  Açıklama: Hesaba giriş sorunu yaşanıyor; müşteri şifresini unuttu ve şifre sıfırlama linki gelmiyor. Müşteri linkin spam klasörünü kontrol etmiş durumda; sonuç negatif. Şu ana kadar sorun için somut aksiyon (yeniden link gönderimi, hesap doğrulama, alternatif yöntem) alınmadı; müşteri hâlâ çözüme kavuşmuş değil ve durum can sıkıcı olarak ifade edildi. Önerilen sonraki adımlar: 1) Şifre sıfırlama linkinin müşterinin hesabına yeniden gönderilmesini sağlamak ve teslimat durumunu kontrol etmek; 2) E-posta adresinin doğruluğunu teyit etmek

## 13. Sonuçları Kaydetme

In [ ]:
# Analiz sonuçlarını kaydet
analysis_results = {
    'timestamp': datetime.now().isoformat(),
    'model_used': analyzer.model,
    'method': 'structured_json_schema_pydantic',
    'sentiment_analysis': {
        'results': sentiment_results,
        'accuracy': sentiment_accuracy,
        'total_analyzed': len(sentiment_results)
    },
    'intent_analysis': {
        'results': intent_results,
        'accuracy': intent_accuracy,
        'total_analyzed': len(intent_results)
    },
    'resolution_analysis': {
        'results': resolution_results,
        'accuracy': resolution_accuracy,
        'total_analyzed': len(resolution_results)
    }
}

# Sonuçları kaydet
with open('openai_analysis_results_structured.json', 'w', encoding='utf-8') as f:
    json.dump(analysis_results, f, ensure_ascii=False, indent=2)

print("✅ Analiz sonuçları 'openai_analysis_results_structured.json' dosyasına kaydedildi")

✅ Analiz sonuçları 'openai_analysis_results_structured.json' dosyasına kaydedildi


## 14. Özet ve Değerlendirme

In [ ]:
print("🎉 STRUCTURED JSON SCHEMA İLE OPENAI API ENTEGRASYONİ TAMAMLANDI!")
print("=" * 70)
print(f"📊 Analiz Özeti:")
print(f"  Model: {analyzer.model}")
print(f"  Yöntem: Structured JSON Schema (Pydantic)")
print(f"  Sentiment Doğruluk: %{sentiment_accuracy:.1f}")
print(f"  Intent Doğruluk: %{intent_accuracy:.1f}")
print(f"  Çözüm Durumu Doğruluk: %{resolution_accuracy:.1f}")
print(f"  Toplam API Çağrısı: {len(sentiment_results) + len(intent_results) + len(resolution_results)}")

print("\n🎯 Avantajlar:")
print("  ✅ Boş çıktı problemi çözüldü")
print("  ✅ JSON parsing hatası yok")
print("  ✅ Otomatik schema validation")
print("  ✅ Daha güvenilir sonuçlar")

print("\n🚀 Sonraki Adımlar:")
print("  1. Daha fazla veri ile test")
print("  2. Prompt optimization")
print("  3. Model performans karşılaştırması")
print("  4. Production deployment")

print("\n📈 Performans Karşılaştırması:")
print("  Eski Yöntem: Manuel parsing, boş çıktılar, %20 sentiment accuracy")
print(f"  Yeni Yöntem: Structured output, %{sentiment_accuracy:.1f} sentiment accuracy")

🎉 STRUCTURED JSON SCHEMA İLE OPENAI API ENTEGRASYONİ TAMAMLANDI!
📊 Analiz Özeti:
  Model: gpt-5-nano
  Yöntem: Structured JSON Schema (Pydantic)
  Sentiment Doğruluk: %80.0
  Intent Doğruluk: %100.0
  Çözüm Durumu Doğruluk: %100.0
  Toplam API Çağrısı: 11

🎯 Avantajlar:
  ✅ Boş çıktı problemi çözüldü
  ✅ JSON parsing hatası yok
  ✅ Otomatik schema validation
  ✅ Daha güvenilir sonuçlar

🚀 Sonraki Adımlar:
  1. Daha fazla veri ile test
  2. Prompt optimization
  3. Model performans karşılaştırması
  4. Production deployment

📈 Performans Karşılaştırması:
  Eski Yöntem: Manuel parsing, boş çıktılar, %20 sentiment accuracy
  Yeni Yöntem: Structured output, %80.0 sentiment accuracy
